<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-1">Load Data</a></span></li><li><span><a href="#Beautiful-Soup" data-toc-modified-id="Beautiful-Soup-2">Beautiful Soup</a></span></li></ul></div>

# Web Scraper
- Beautiful Soup: https://realpython.com/beautiful-soup-web-scraper-python/

In [1]:
import pandas as pd
import time, datetime
import multiprocessing as mp
from datetime import datetime as dt
import math

## Load Data

In [ ]:
bad_cols = [0,2,6,15,16,17,18,19,20,21,22,24,25,26] # unnecessary columns from example_data notebook
cols = list(range(0,26)) # list of all columns
good_cols = [i for i in cols if i not in bad_cols] # create list of useful columns
good_cols = [str(x) for x in good_cols] # convert to string
good_cols

In [3]:
start = time.time()
# read data in chunks of 100k rows at a time
chunk = pd.read_csv('../../00-data/gdelt_data/2021/gdelt_sum_dec2021.csv.zip', 
                    chunksize=100000, 
                    dtype={'3': 'str'})#, # mixed data types
                    #index_col=[0]) # specify index column
# iter_csv = pd.read_csv('file.csv', iterator=True, chunksize=1000)
# df = pd.concat([chunk[chunk['field'] > constant] for chunk in iter_csv])
df = pd.concat(chunk) # concat chunks together
end = time.time()
print("Read csv with chunks: ",(end-start),"sec")

Read csv with chunks:  10.28976583480835 sec


In [4]:
df

,1,3,4,5,7,8,9,10,11,12,13,14,23
0,20211201000000,mcdowellnews.com,https://mcdowellnews.com/news/national/govt-an...,NaN,NaN,NaN,"2#Alaska, United States#US#USAK#61.385#-152.26...","2#Alaskans, United States#US#USAK##61.385#-152...",mike dunleavy;christine oconnor,"Mike Dunleavy,586;Christine Oconnor,256",alaska telecom association,"Alaska Telecom Association,205","Alaska Telecom Association,225;Mike Dunleavy,616"
1,20211201000000,observer.ug,https://www.observer.ug/viewpoint/72021-why-sh...,NaN,GENERAL_GOVERNMENT;EPU_POLICY;EPU_POLICY_GOVER...,"WB_178_PEST_MANAGEMENT,4266;WB_174_CROP_PRODUC...",1#Uganda#UG#UG#2#33#UG,1#Uganda#UG#UG##2#33#UG#204;1#Uganda#UG#UG##2#...,isabella akiteng,"Isabella Akiteng,949",friedrich ebert stiftung youth leaders program...,Friedrich Ebert Stiftung Youth Leaders Program...,"Friedrich Ebert Stiftung,902;Youth Leaders,917..."
2,20211201000000,menafn.com,https://menafn.com/1103282515/9-injured-in-ref...,WOUND#9##1#Algeria#AG#AG#28#3#AG;CRISISLEX_CRI...,WOUND;CRISISLEX_CRISISLEXREC;CRISISLEX_C03_WEL...,"LEADER,524;ENV_OIL,96;ENV_MINING,539;DISASTER_...","4#Skikda, Wilaya De Skikda, Algeria#AG#AG31#36...","4#Skikda, Wilaya De Skikda, Algeria#AG#AG31#38...",mohamed arkab,"Mohamed Arkab,553",sonatrach;menafn- trend news agency;sonatrach ...,"Sonatrach,295;Sonatrach,344;Sonatrach,714;Mena...","News Agency,23;Sonatrach Group,299;Algerian Mi..."
3,20211201000000,hot1063.com,https://www.hot1063.com/news/red-notice-is-now...,NaN,CRIME_COMMON_ROBBERY;TAX_FNCACT;TAX_FNCACT_THI...,"CRIME_COMMON_ROBBERY,727;TAX_FNCACT_THIEF,727;...",NaN,NaN,gal gadot;ryan reynolds;dwayne johnson,"Gal Gadot,417;Ryan Reynolds,405;Dwayne Johnson,14",netflix;warnermedia company;cable news network...,"Netflix,147;Netflix,251;Netflix,607;Warnermedi...","Dwayne Johnson,15;Sandra Bullock,340;Ryan Reyn..."
4,20211201000000,altenergymag.com,https://www.altenergymag.com/company_directory...,NaN,ENV_SOLAR;,"ENV_SOLAR,75;ENV_SOLAR,313;ENV_SOLAR,353;ENV_S...",1#United States#US#US#39.828175#-98.5795#US,1#America#US#US##39.828175#-98.5795#US#626;1#U...,NaN,NaN,company sector;free company listing;company ca...,"Company Sector,785;Free Company Listing,901;Co...","Project Development Support,94;Pima Center Pkw..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
150669,20211201234500,freightwaves.com,https://www.freightwaves.com/news/thanksgiving...,NaN,NEGOTIATIONS;GEN_HOLIDAY;USPEC_POLICY1;EPU_POL...,"GEN_HOLIDAY,523;GEN_HOLIDAY,1080;GEN_HOLIDAY,1...",1#United States#US#US#39.828175#-98.5795#US;3#...,"2#Texas, United States#US#USTX##31.106#-97.647...",los angeles;kevin hill;tony mulvey,"Los Angeles,9113;Los Angeles,12066;Los Angeles...",walmart;freightwaves freight intel group;rate ...,"Walmart,3452;Freightwaves Freight Intel Group,...","Chain Pricing Power Index,46;Chain Pricing Pow..."
150670,20211201234500,newyorktelegraph.com,https://www.newyorktelegraph.com/news/27184719...,NaN,TAX_FNCACT;TAX_FNCACT_WORKERS;USPEC_POLICY1;EP...,"GENERAL_GOVERNMENT,212;EPU_POLICY_GOVERNMENT,2...","1#Germany#GM#GM#51.5#10.5#GM;4#Paris, France (...","4#Petersburg, Sankt-Peterburg, Russia#RS#RS66#...",peng shuai;queen elizabeth,"Peng Shuai,599;Queen Elizabeth,752",women tennis association;international section,"Women Tennis Association,522;International Sec...","Tennis Association,529;Peng Shuai,612;Queen El..."
150671,20211201234500,dailyrecord.co.uk,https://www.dailyrecord.co.uk/lifestyle/warnin...,NaN,TAX_WORLDMAMMALS;TAX_WORLDMAMMALS_DOG;TAX_WORL...,"AGRICULTURE,4612;WB_1467_EDUCATION_FOR_ALL,307...",1#Sweden#SW#SW#62#15#SW;1#United Kingdom#UK#UK...,1#United Kingdom#UK#UK##54#-4#UK#202;1#United ...,carolyn menteith,"Carolyn Menteith,872",NaN,NaN,"United Kingdom,215;Agria Pet,282;Team Dogs,323..."
150672,20211201234500,scmp.com,https://www.scmp.com/news/china/diplomacy/arti...,NaN,GENERAL_HEALTH;MEDICAL;CRISISLEX_C07_SAFETY;UN...,"UNGP_FORESTS_RIVERS_OCEANS,115;GENERAL_HEALTH,...",1#China#CH#CH#35#105#CH,1#China#CH#CH##35#105#CH#23,NaN,NaN,reuters,"R

In [5]:
bad_rows = df[~df['1'].str.get(0).isin(['2'])].index # remove rows with dates that don't start with 2
bad_rows = bad_rows.append(df[~df['1'].str.get(1).isin(['0'])].index) # remove rows with dates second number isn't 0
df = df.drop(index = bad_rows)
df['1'] = [int(float(i)) for i in df['1']]
df

,1,3,4,5,7,8,9,10,11,12,13,14,23
0,20211201000000,mcdowellnews.com,https://mcdowellnews.com/news/national/govt-an...,NaN,NaN,NaN,"2#Alaska, United States#US#USAK#61.385#-152.26...","2#Alaskans, United States#US#USAK##61.385#-152...",mike dunleavy;christine oconnor,"Mike Dunleavy,586;Christine Oconnor,256",alaska telecom association,"Alaska Telecom Association,205","Alaska Telecom Association,225;Mike Dunleavy,616"
1,20211201000000,observer.ug,https://www.observer.ug/viewpoint/72021-why-sh...,NaN,GENERAL_GOVERNMENT;EPU_POLICY;EPU_POLICY_GOVER...,"WB_178_PEST_MANAGEMENT,4266;WB_174_CROP_PRODUC...",1#Uganda#UG#UG#2#33#UG,1#Uganda#UG#UG##2#33#UG#204;1#Uganda#UG#UG##2#...,isabella akiteng,"Isabella Akiteng,949",friedrich ebert stiftung youth leaders program...,Friedrich Ebert Stiftung Youth Leaders Program...,"Friedrich Ebert Stiftung,902;Youth Leaders,917..."
2,20211201000000,menafn.com,https://menafn.com/1103282515/9-injured-in-ref...,WOUND#9##1#Algeria#AG#AG#28#3#AG;CRISISLEX_CRI...,WOUND;CRISISLEX_CRISISLEXREC;CRISISLEX_C03_WEL...,"LEADER,524;ENV_OIL,96;ENV_MINING,539;DISASTER_...","4#Skikda, Wilaya De Skikda, Algeria#AG#AG31#36...","4#Skikda, Wilaya De Skikda, Algeria#AG#AG31#38...",mohamed arkab,"Mohamed Arkab,553",sonatrach;menafn- trend news agency;sonatrach ...,"Sonatrach,295;Sonatrach,344;Sonatrach,714;Mena...","News Agency,23;Sonatrach Group,299;Algerian Mi..."
3,20211201000000,hot1063.com,https://www.hot1063.com/news/red-notice-is-now...,NaN,CRIME_COMMON_ROBBERY;TAX_FNCACT;TAX_FNCACT_THI...,"CRIME_COMMON_ROBBERY,727;TAX_FNCACT_THIEF,727;...",NaN,NaN,gal gadot;ryan reynolds;dwayne johnson,"Gal Gadot,417;Ryan Reynolds,405;Dwayne Johnson,14",netflix;warnermedia company;cable news network...,"Netflix,147;Netflix,251;Netflix,607;Warnermedi...","Dwayne Johnson,15;Sandra Bullock,340;Ryan Reyn..."
4,20211201000000,altenergymag.com,https://www.altenergymag.com/company_directory...,NaN,ENV_SOLAR;,"ENV_SOLAR,75;ENV_SOLAR,313;ENV_SOLAR,353;ENV_S...",1#United States#US#US#39.828175#-98.5795#US,1#America#US#US##39.828175#-98.5795#US#626;1#U...,NaN,NaN,company sector;free company listing;company ca...,"Company Sector,785;Free Company Listing,901;Co...","Project Development Support,94;Pima Center Pkw..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
150669,20211201234500,freightwaves.com,https://www.freightwaves.com/news/thanksgiving...,NaN,NEGOTIATIONS;GEN_HOLIDAY;USPEC_POLICY1;EPU_POL...,"GEN_HOLIDAY,523;GEN_HOLIDAY,1080;GEN_HOLIDAY,1...",1#United States#US#US#39.828175#-98.5795#US;3#...,"2#Texas, United States#US#USTX##31.106#-97.647...",los angeles;kevin hill;tony mulvey,"Los Angeles,9113;Los Angeles,12066;Los Angeles...",walmart;freightwaves freight intel group;rate ...,"Walmart,3452;Freightwaves Freight Intel Group,...","Chain Pricing Power Index,46;Chain Pricing Pow..."
150670,20211201234500,newyorktelegraph.com,https://www.newyorktelegraph.com/news/27184719...,NaN,TAX_FNCACT;TAX_FNCACT_WORKERS;USPEC_POLICY1;EP...,"GENERAL_GOVERNMENT,212;EPU_POLICY_GOVERNMENT,2...","1#Germany#GM#GM#51.5#10.5#GM;4#Paris, France (...","4#Petersburg, Sankt-Peterburg, Russia#RS#RS66#...",peng shuai;queen elizabeth,"Peng Shuai,599;Queen Elizabeth,752",women tennis association;international section,"Women Tennis Association,522;International Sec...","Tennis Association,529;Peng Shuai,612;Queen El..."
150671,20211201234500,dailyrecord.co.uk,https://www.dailyrecord.co.uk/lifestyle/warnin...,NaN,TAX_WORLDMAMMALS;TAX_WORLDMAMMALS_DOG;TAX_WORL...,"AGRICULTURE,4612;WB_1467_EDUCATION_FOR_ALL,307...",1#Sweden#SW#SW#62#15#SW;1#United Kingdom#UK#UK...,1#United Kingdom#UK#UK##54#-4#UK#202;1#United ...,carolyn menteith,"Carolyn Menteith,872",NaN,NaN,"United Kingdom,215;Agria Pet,282;Team Dogs,323..."
150672,20211201234500,scmp.com,https://www.scmp.com/news/china/diplomacy/arti...,NaN,GENERAL_HEALTH;MEDICAL;CRISISLEX_C07_SAFETY;UN...,"UNGP_FORESTS_RIVERS_OCEANS,115;GENERAL_HEALTH,...",1#China#CH#CH#35#105#CH,1#China#CH#CH##35#105#CH#23,NaN,NaN,reuters,"R

In [6]:
filter_df = df[df['3']=='reuters.com']
filter_df

,1,3,4,5,7,8,9,10,11,12,13,14,23
471,20211201000000,reuters.com,https://www.reuters.com/world/us/us-labor-secr...,NaN,TAX_FNCACT;TAX_FNCACT_SECRETARY;ECON_UNIONS;TA...,"IDEOLOGY,2705;TAX_FNCACT_DOCKWORKERS,269;TAX_F...","2#California, United States#US#USCA#36.17#-119...","3#Los Angeles, California, United States#US#US...",lisa baertlein;los angeles;richard pullin;mart...,"Lisa Baertlein,2592;Los Angeles,11;Los Angeles...",thomson reuters trust principles;walmart;pacif...,"Thomson Reuters Trust Principles,2705;Walmart,...","Southern California,158;Long Beach,372;Secreta..."
2401,20211201001500,reuters.com,https://www.reuters.com/markets/currencies/bit...,NaN,ECON_BITCOIN;TAX_FNCACT;TAX_FNCACT_MINER;ECON_...,"IDEOLOGY,1444;ECON_STOCKMARKET,1251;TAX_FNCACT...","4#Bengaluru, Karnataka, India#IN#IN19#12.9833#...","4#Bengaluru, Karnataka, India#IN#IN19#70159#12...",mehnaz yasmin;radhika anilkumar;aditya soni,"Mehnaz Yasmin,1324;Radhika Anilkumar,1344;Adit...",thomson reuters trust principles;york stock ex...,"Thomson Reuters Trust Principles,1444;York Sto...","Griid Infrastructure,59;Adit EdTech Acquisitio..."
3556,20211201003000,reuters.com,https://www.reuters.com/world/asia-pacific/us-...,NaN,TAX_FNCACT;TAX_FNCACT_REPRESENTATIVE;TAX_WORLD...,"IDEOLOGY,1587;GENERAL_GOVERNMENT,758;EPU_POLIC...","1#Afghanistan#AF#AF#33#66#AF;3#Washington, Was...",1#Afghanistan#AF#AF##33#66#AF#106;1#Afghanista...,mohammad zargham;sam holmes;kanishka singh;jon...,"Mohammad Zargham,1474;Sam Holmes,1532;Kanishka...",thomson reuters trust principles;treasury depa...,"Thomson Reuters Trust Principles,1587;Treasury...","Thomas West,115;Afghan Taliban,158;State Depar..."
3808,20211201003000,reuters.com,https://www.reuters.com/world/europe/france-se...,NaN,SECURITY_SERVICES;TAX_FNCACT;TAX_FNCACT_POLICE...,"IDEOLOGY,1439;GENERAL_GOVERNMENT,257;EPU_POLIC...","1#Martinique#MB#MB#14.666667#-61#MB;4#Paris, F...",1#French#FR#FR##46#2#FR#70;1#Martinique#MB#MB#...,jean terzian;sebastien lecornu;richard lough;a...,"Jean Terzian,1357;Sebastien Lecornu,443;Richar...",thomson reuters trust principles;reuters,"Thomson Reuters Trust Principles,1439;Reuters,...","French Caribbean,85;Richard Lough,1374;Jean Te..."
4018,20211201003000,reuters.com,https://www.reuters.com/world/europe/greece-re...,KILL#19#related#1#Greece#GR#GR#39#22#GR;,TAX_DISEASE;TAX_DISEASE_COVID-19;GENERAL_GOVER...,"IDEOLOGY,2293;GENERAL_GOVERNMENT,163;GENERAL_G...","4#Athens, AttikÃ­R, Greece#GR#GR35#37.9833#23....",1#Greeks#GR#GR##39#22#GR#1264;1#Greece#GR#GR##...,adonis georgiadis;kyriakos mitsotakis;george g...,"Adonis Georgiadis,646;Kyriakos Mitsotakis,1600...",thomson reuters trust principles;reuters,"Thomson Reuters Trust Principles,2293;Reuters,...","Minister Adonis Georgiadis,674;Prime Minister ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
149173,20211201234500,reuters.com,https://jp.reuters.com/jp.reuters.com/news/pic...,NaN,GENERAL_GOVERNMENT;EPU_POLICY;EPU_POLICY_GOVER...,"GENERAL_GOVERNMENT,78;EPU_POLICY_GOVERNMENT,78...","1#Mexico#MX#MX#23#-102#MX;4#Huehuetan, Chiapas...","4#Huehuetan, Chiapas, Mexico#MX#MX05#19688#15....",jose luis gonzalez,"Jose Luis Gonzalez,262",reuters,"Reuters,7;Reuters,243","Jose Luis,301"
149384,20211201234500,reuters.com,https://www.reuters.com/markets/commodities/cn...,NaN,TAX_ETHNICITY;TAX_ETHNICITY_CANADIAN;WB_135_TR...,"IDEOLOGY,1031;TRAFFIC,579;NATURAL_DISASTER_FLO...","4#Kamloops, British Columbia, Canada#CA#CA02#5...","4#Winnipeg, Manitoba, Canada#CA#CA03#12577#49....",leslie adler;ismail shakil,"Leslie Adler,976;Ismail Shakil,897",thomson reuters trust principles;canadian paci...,"Thomson Reuters Trust Principles,1031;Canadian...","British Columbia,213;British Columbia,433;Prin..."
149497,20211201234500,reuters.com,https://www.reuters.com/business/healthcare-ph...,WOUND#1##1#United States#US#US#39.828175#-98.5...,TAX_DISEASE;TAX_DISEASE_COVID-19;TAX_FNCACT;TA...,"IDEOLOGY,4497;TAX_DISEASE_HEMORRHAGE,3394;TAX_...",1#United States#US#US#39.828175

In [7]:
filter_urls = filter_df['4']
filter_urls

471       https://www.reuters.com/world/us/us-labor-secr...
2401      https://www.reuters.com/markets/currencies/bit...
3556      https://www.reuters.com/world/asia-pacific/us-...
3808      https://www.reuters.com/world/europe/france-se...
4018      https://www.reuters.com/world/europe/greece-re...
                                ...                        
149173    https://jp.reuters.com/jp.reuters.com/news/pic...
149384    https://www.reuters.com/markets/commodities/cn...
149497    https://www.reuters.com/business/healthcare-ph...
150065    https://www.reuters.com/markets/us/singapore-r...
150602    https://www.reuters.com/business/cop/dont-blam...
Name: 4, Length: 350, dtype: object

In [8]:
filter_urls.iloc[1]

'https://www.reuters.com/markets/currencies/bitcoin-miner-griid-infrastructure-go-public-via-33-bln-spac-deal-2021-11-30/'

## Beautiful Soup

In [24]:
from bs4 import BeautifulSoup
import requests

URL = filter_urls.iloc[1]
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

In [25]:
soup

<!DOCTYPE html>
<html lang="en"><head><title>Bitcoin miner Griid Infrastructure to go public via $3.3 bln SPAC deal | Reuters</title><meta content="width=device-width, initial-scale=1" name="viewport"/><script>(function(){
      var current_location = window.location.href;

      if (current_location.indexOf('/info-pages/supported-browsers/') === -1) {
        var supportFetchApi = 'fetch' in window;
        var supportCSSGrid = window.CSS && CSS.supports('display', 'grid');

        if (!supportFetchApi && !supportCSSGrid) {
          window.location.href = '/info-pages/supported-browsers/';
        }
      }
    })()</script><script async="" data-config='{"ADMIN":false,"SEGMENT_WRITE_KEY":"IEWBqQ8VWHijTQxb7lEBGFGS9uIJzigZ","SEGMENT_WRITE_KEY_MOBILE":"YlmAIaFBxsNtlVJdfuSV0ncE931ghRtS","API_ORIGIN":"https://api-reuters-reuters-prod.cdn.arcpublishing.com","ONETRUST_SCRIPT_ID":"38cb75bd-fbe1-4ac8-b4af-e531ab368caf","SOPHI_SALT":"164aybEpKnoJKEgoB5iIwzy9q01Lg2vz","SOPHI_PAGES":["A","/worl

In [26]:
master = soup.find(id="fusion-app")
print(master.prettify())

<div class="fusion-app" id="fusion-app">
 <div class="regular-article-layout__container___1O5R7C">
  <div class="regular-article-layout__header___AY0AiN">
   <header>
    <a class="Button__link___iJKfD7 Button__secondary___1qNGNf Button__round___SR4lqT TextButton__container___2ipwLP SkipToLink__skiptolink___3TTRlQ skiptolink" href="#main-content">
     <div class="Button__container___3WEvvw">
      <span class="Text__text___2vZwzq Text__inherit-color___1Jv4Eu Text__bold___2pIB3c Text__default___1ZhUwy TextButton__medium___1HuzMG">
       Skip to main content
      </span>
     </div>
    </a>
    <a class="Button__link___iJKfD7 Button__secondary___1qNGNf Button__round___SR4lqT TextButton__container___2ipwLP SkipToLink__skiptolink___3TTRlQ skiptolink" href="#mini-floating-vid">
     <div class="Button__container___3WEvvw">
      <span class="Text__text___2vZwzq Text__inherit-color___1Jv4Eu Text__bold___2pIB3c Text__default___1ZhUwy TextButton__medium___1HuzMG">
       Skip to floating m

In [27]:
final = master.find("div", class_="article-body__content___ROHHzW paywall-article")
print(final.text)

Nov 30 (Reuters) - Bitcoin miner Griid Infrastructure said on Tuesday it would go public by merging with blank-check firm Adit EdTech Acquisition Corp (ADEX.N) in a deal valuing the combined company at about $3.3 billion, including debt.Ohio-based Griid, the latest bitcoin mining startup to take the blank-check route, will receive $246 million in cash from Adit EdTech's trust account after the deal closes.A surge in digital currencies this year and increasing backing from major companies have propelled startups in the sector to sky-high valuations in recent months, even as the industry faces scrutiny from regulators.Register now for FREE unlimited access to Reuters.comRegisterEarlier this month, cryptocurrency miner Bitdeer Technologies agreed to merge with special-purpose acquisition company (SPAC) Blue Safari Group Acquisition (BSGA.O) in a deal valued at $4 billion. read more That deal followed another cryptocurrency miner Core Scientific Holding Co's announcement in July of a merge

In [16]:
sum_urls = filter_urls[0:10]
sum_urls

471     https://www.reuters.com/world/us/us-labor-secr...
2401    https://www.reuters.com/markets/currencies/bit...
3556    https://www.reuters.com/world/asia-pacific/us-...
3808    https://www.reuters.com/world/europe/france-se...
4018    https://www.reuters.com/world/europe/greece-re...
4039    https://www.reuters.com/article/us-health-coro...
4208    https://www.reuters.com/article/us-health-coro...
4596    https://www.reuters.com/world/us/celebrity-sur...
5553    https://www.reuters.com/legal/government/judge...
6570    https://www.reuters.com/markets/commodities/us...
Name: 4, dtype: object

In [29]:
from bs4 import BeautifulSoup
import requests
ls = []
for i in range(len(sum_urls)): #filter_urls
    URL = sum_urls.iloc[i] # filter_urls
    page = requests.get(URL,headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'})
    soup = BeautifulSoup(page.content, "html.parser")
    master = soup.find(id="fusion-app")
    final = master.find("div", class_="article-body__content___ROHHzW paywall-article")
    print(URL)
    print(final.text)

https://www.reuters.com/world/us/us-labor-secretary-visits-top-west-coast-ports-union-talks-loom-2021-11-30/
LOS ANGELES, Nov 30 (Reuters) - The U.S. labor secretary on Tuesday met with unions and employers at the nation's busiest port complex in Southern California as anxiety builds ahead of next year's contract negotiations covering roughly 22,000 West Coast dockworkers.The contract expires in July, and the stakes are particularly high at the Los Angeles/Long Beach port complex that handles some 40% of U.S. imports. Docks at the side-by-side ports are crowded with cargo boxes and more than four dozen container ships are waiting to unload as retailers like Walmart scramble to stock shelves with holiday gifts.The last round of talks in 2014 and 2015 were contentious, leading to temporary port shutdowns. A repeat of that could harm a U.S. economy already grappling with inflation fueled by soaring shipping costs and product shortages.Register now for FREE unlimited access to Reuters.comR

https://www.reuters.com/article/us-health-coronavirus-canada/canada-to-ask-air-arrivals-to-take-covid-tests-except-on-u-s-flights-idUSKBN2IF2AP
OTTAWA, Nov 30 (Reuters) - Canada, seeking to halt the spread of the Omicron coronavirus variant, will require people arriving by air from all nations except the United States to take a COVID-19 test, Health Minister Jean-Yves Duclos said on Tuesday.Ottawa is also expanding a ban on travelers from southern Africa to cover three more nations, bringing the total to 10.Canada has identified seven people with the new variant, at least four of whom were recently in Nigeria.Register now for FREE unlimited access to Reuters.comRegisterThe Canadian government also said it was in advanced talks with Pfizer Inc (PFE.N) and Merck & Co Inc (MRK.N) regarding a purchase agreement for COVID-19 antiviral drugs."All air travelers coming from outside Canada, apart from the United States, will now need to be tested at the airport (where) they are landing in Canad

https://www.reuters.com/markets/commodities/us-energy-chief-hints-california-may-grant-reprieve-its-last-nuclear-plant-2021-11-30/
WASHINGTON, Nov 30 (Reuters) - California could reconsider closing its last nuclear power plant as public support has grown for the low-carbon energy source, U.S. Energy Secretary Jennifer Granholm told Reuters on Tuesday.She said she was willing to eventually talk with state officials about keeping the Diablo Canyon plant open.The Biden administration has expressed support for the nuclear power industry as crucial to its goal of decarbonizing the U.S. electrical grid by 2035.Register now for FREE unlimited access to Reuters.comRegister"California has been very bullish on zero-carbon emission energy," Granholm said in a wide-ranging interview to be broadcast next week at the Reuters Events conference Energy Transition North America 2021, where leaders will discuss the move to clean energy."It may be something that they decide to take a look at, given that I